In [87]:
import numpy as np
import gym
import gym_chess
import chess
import os


env = gym.make('ChessAlphaZero-v0')


In [75]:
#NOTE to remember:
# board = chess.Board(fen)  #set board to a specific position from fen encoding
# might have to reset env if encoding or such does not work

In [188]:
#encoding from gym-chess
env = gym.make('ChessAlphaZero-v0')
env.reset()
print(env.observation_space)
print(env.action_space)

Box(0, 2147483647, (8, 8, 119), int32)
Discrete(4672)


In [190]:
#gym-chess environment:


#encode:
move = chess.Move.from_uci('e2e4')
env.encode(move)

#decode:
env.decode(877)


# env.legal_actions #moves with encoded notation
# env.legal_moves #moves with decoded notation

Move.from_uci('e2e4')

In [143]:
#encoding func from alpha zero:


def encodeBoardFromFen(fen: str) -> np.array:
	board = chess.Board(fen)
	return encodeBoard(board)
	
def encodeBoard(board: chess.Board) -> np.array:
	"""Converts a board to numpy array representation."""

	array = np.zeros((8, 8, 14), dtype=int)

	for square, piece in board.piece_map().items():
		rank, file = chess.square_rank(square), chess.square_file(square)
		piece_type, color = piece.piece_type, piece.color
	
		# The first six planes encode the pieces of the active player, 
		# the following six those of the active player's opponent. Since
		# this class always stores boards oriented towards the white player,
		# White is considered to be the active player here.
		offset = 0 if color == chess.WHITE else 6
		
		# Chess enumerates piece types beginning with one, which we have
		# to account for
		idx = piece_type - 1
	
		array[rank, file, idx + offset] = 1

	# Repetition counters
	array[:, :, 12] = board.is_repetition(2)
	array[:, :, 13] = board.is_repetition(3)

	return array

def encodeMove(move: str):
	move = chess.Move.from_uci(move) #could reset but increases time with over 50%
	return env.encode(move)

def decodeMove(move: int):
	return env.decode(move)


In [164]:
#function to encode all moves and positions from rawData folder
def encodeAllMovesAndPositions():
    env = gym.make('ChessAlphaZero-v0')
    env.reset()
    #find all files in folder:

    files = os.listdir('data/rawData')
    for idx, f in enumerate(files):
        movesAndPositions = np.load(f'data/rawData/{f}', allow_pickle=True)
        moves = movesAndPositions[:,0]
        positions = movesAndPositions[:,1]
        encodedMoves = []
        encodedPositions = []
        for i in range(len(moves)):
            try:
                encodedMoves.append(encodeMove(moves[i])) 
                encodedPositions.append(encodeBoardFromFen(positions[i]))
                pass
            except:
                print(f'error in file: {f}')
                print(moves[i])
                print(positions[i])
                print(i)
                return
            
        np.save(f'data/preparedData/moves{idx}', np.array(encodedMoves))
        np.save(f'data/preparedData/positions{idx}', np.array(encodedPositions))
    
encodeAllMovesAndPositions()

#NOTE: shape of files:
#moves: (number of moves in gamew)
#positions: (number of moves in game, 8, 8, 14) (number of moves in game is including both black and white moves)

(114,)